In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sklearn.preprocessing import OneHotEncoder
import config

In [2]:
df = pd.read_csv("./Resources/corporate_rating.csv")
df.head()

,Rating,Name,Symbol,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,...,effectiveTaxRate,freeCashFlowOperatingCashFlowRatio,freeCashFlowPerShare,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover
0,A,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,Consumer Durables,0.945894,0.426395,0.099690,44.203245,...,0.202716,0.437551,6.810673,9.809403,4.008012,0.049351,7.057088,15.565438,0.058638,3.906655
1,BBB,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,Consumer Durables,1.033559,0.498234,0.203120,38.991156,...,0.074155,0.541997,8.625473,17.402270,3.156783,0.048857,6.460618,15.914250,0.067239,4.002846
2,BBB,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,Consumer Durables,0.963703,0.451505,0.122099,50.841385,...,0.214529,0.513185,9.693487,13.103448,4.094575,0.044334,10.491970,18.888889,0.074426,3.483510
3,BBB,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,Consumer Durables,1.019851,0.510402,0.176116,41.161738,...,1.816667,-0.147170,-1.015625,14.440104,3.630950,-0.012858,4.080741,6.901042,0.028394,4.581150
4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,Consumer Durables,0.957844,0.495432,0.141608,47.761126,...,0.166966,0.451372,7.135348,14.257556,4.012780,0.053770,8.293505,15.808147,0.058065,3.857790


In [3]:
#Cleaning
target = df['Rating']
df.drop(['Name','Date','Rating','Symbol'],axis = 1, inplace=True)


In [4]:
#Check how many uniqye agency names/sectors there are
df['Sector'].nunique()

12

In [5]:
df['Rating Agency Name'].nunique()

5

In [6]:
df['Sector'].value_counts() 

Energy                   294
Basic Industries         260
Consumer Services        250
Technology               234
Capital Goods            233
Public Utilities         211
Health Care              171
Consumer Non-Durables    132
Consumer Durables         74
Transportation            63
Miscellaneous             57
Finance                   50
Name: Sector, dtype: int64

In [7]:
#Need to group up Sectors

to_replace = df['Sector'].value_counts() < 100
to_replace = to_replace[to_replace].index.tolist()


# Replace in dataframe
for a in to_replace:
    df['Sector'] = df['Sector'].replace(a,"Other")

#Check to make sure binning was successful
df['Sector'].value_counts()

Energy                   294
Basic Industries         260
Consumer Services        250
Other                    244
Technology               234
Capital Goods            233
Public Utilities         211
Health Care              171
Consumer Non-Durables    132
Name: Sector, dtype: int64

In [8]:
encoded_df = pd.get_dummies(df, columns = ['Rating Agency Name','Sector'])
encoded_df.head()

,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,netProfitMargin,pretaxProfitMargin,grossProfitMargin,operatingProfitMargin,returnOnAssets,returnOnCapitalEmployed,...,Rating Agency Name_Standard & Poor's Ratings Services,Sector_Basic Industries,Sector_Capital Goods,Sector_Consumer Non-Durables,Sector_Consumer Services,Sector_Energy,Sector_Health Care,Sector_Other,Sector_Public Utilities,Sector_Technology
0,0.945894,0.426395,0.099690,44.203245,0.037480,0.049351,0.176631,0.061510,0.041189,0.091514,...,0,0,0,0,0,0,0,1,0,0
1,1.033559,0.498234,0.203120,38.991156,0.044062,0.048857,0.175715,0.066546,0.053204,0.104800,...,0,0,0,0,0,0,0,1,0,0
2,0.963703,0.451505,0.122099,50.841385,0.032709,0.044334,0.170843,0.059783,0.032497,0.075955,...,0,0,0,0,0,0,0,1,0,0
3,1.019851,0.510402,0.176116,41.161738,0.020894,-0.012858,0.138059,0.042430,0.025690,-0.027015,...,0,0,0,0,0,0,0,1,0,0
4,0.957844,0.495432,0.141608,47.761126,0.042861,0.053770,0.177720,0.065354,0.046363,0.096945,...,1,0,0,0,0,0,0,1,0,0


In [9]:
#Need to re-encode Target variable as a numerical
#Setting AAA = 0, AA = 1, etc...
alternate_target = target.replace(['AAA','AA','A','BBB','BB','B','CCC','CC','C','D'],[1,1,1,0,0,0,0,0,0,0])
target.replace(['AAA','AA','A','BBB','BB','B','CCC','CC','C','D'],range(0,10), inplace=True)


In [10]:
#Save outputs to CSV - PLACEHOLDER UNTIL DATABASE IS SETUP
encoded_df.to_csv('./Resources/cleaned_placeholder_X.csv')
target.to_csv('./Resources/cleaned_y.csv')
alternate_target.to_csv('./Resources/alternate_y.csv')


In [11]:
#Connect and save to a local Postgres server
#Requires a server to be running on your machine 
protocol = 'postgresql'
username = config.username
password = config.password
host = 'localhost'
port = 5432
database_name = config.database_name
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
con = engine.connect()

In [ ]:
encoded_df.to_sql('X',con,if_exists='replace')
target.to_sql('y',con,if_exists='replace')
pd.read_csv("./Resources/corporate_rating.csv").to_sql('original',con,if_exists='replace')